# ***This is a simple implement of basic 1st attack introduced in ML-leak article on CIFAR10***

## ***General setting code***

### ***Install all independency here***

In [ ]:
!pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

### ***Import all dependency here***

In [ ]:
# to monitor the progress
from tqdm import tqdm
import time
# basic dependency
import numpy as np
import random
# pytorch related
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import optim
import torch.nn.functional as F
import torchvision.transforms as T
# for visulization
import matplotlib.pyplot as plt
from random import shuffle
# sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# other classifier
import lightgbm as lgb
# for mount from Colab to my drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive


## ***Target and shadow model code***

### ***Define a Neural Network class for shadow and target model***

In [ ]:
# reuse the NN in badnet
class basicNet(nn.Module):

    def __init__(self,inputchannels,outputclasses):
        super().__init__()
        self.conv1 = nn.Conv2d(inputchannels, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool = nn.AvgPool2d(2)
        if inputchannels == 3:
          inputfeatures = 800
        else:
          inputfeatures = 512
        self.fc1 = nn.Linear(inputfeatures, 512)
        self.fc2 = nn.Linear(512, outputclasses)

    def forward(self, x):
        # conv block1
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        # conv block2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        # reshape(flat) the feature to be the input of full connect layer
        x = x.view(-1, self.num_features(x))
        # fc block1
        x = self.fc1(x)
        x = F.relu(x)
        # fc block2
        x = self.fc2(x)
        x = F.softmax(x,dim=-1)
        return x

    def num_features(self, x):
        # size of different dimensions
        size_D = x.size()[1:]
        total = 1
        for i in size_D:
            total = total*i
        return total

### ***Functions for training and evaluating(target/shadow model)***

In [ ]:
def train(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in tqdm(enumerate(dataloader)):
        opt.zero_grad()
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        predict = model(imgs)
        #print(predict.dtype)
        #print(labels.dtype)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count


def evaluation(model, dataloader, batch_size=64):
    # switch to model:eval
    # no difference here since no dropout and BN    
    model.eval()
    # y_tensorlist is a list consists of some tensors
    y_true_tensorlist = []
    y_predict_tensorlist = []
    for step, (batch_x, batch_y) in enumerate(dataloader):
        batch_x,batch_y = batch_x.to(device),batch_y.to(device)
        output = model(batch_x)
        #print(output.shape)
        batch_y_predict = torch.argmax(output, dim=1)
        
        y_predict_tensorlist.append(batch_y_predict)
        
        y_true_tensorlist.append(batch_y)
        
    # combine the tensors in the list into one
    y_true = torch.cat(y_true_tensorlist,0)
    y_predict = torch.cat(y_predict_tensorlist,0)

    # compute accuracy
    length = len(y_true)
    right_length = torch.sum(y_true == y_predict)
    #print(right_length/length)
    
    return right_length/length

## ***Attack model's dataset code***

### ***Functions used to create attack model's dataset***

In [ ]:
def attackdataset_x_1_batch(shadow,batch_x,device):
    shadow.eval()
    # Aim to get features(x)
    # output of model shadow is probabilities
    batch_y = shadow(batch_x.to(device))
    sorted_prob,_ = torch.sort(batch_y,descending=True)
    #print(sorted_prob.shape)

    # choose 3 highest probabilities as feature vector
    # This is a batch's output, not single image.
    # feature: x, x is already a tensor
    feature_vectors = torch.narrow(sorted_prob,dim=1,start=0,length=3)

    return feature_vectors

def get_attackdataset(shadow,train_loader,out_loader,device):
    shadow.eval()
    # 1. get a tensor list of x
    # 2. combine all tensors in tensor list x into a tensor X
    # 3. generate label tensor Y, the length is the same as X
    attack_set=[]

    attack_X_train_list = []
    for i, (batch_x, batch_y) in enumerate(train_loader):
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        feature_vectors = attackdataset_x_1_batch(shadow,batch_x,device=device)
        attack_X_train_list.append(feature_vectors)
    attack_X_train = torch.cat(attack_X_train_list,0)
    print(len(attack_X_train))
    for i in attack_X_train:
      attack_set.append((i.detach(),torch.ones(1)))

    # train:label = 1
    #attack_Y_train = torch.ones(attack_X_train.shape[0])
    #print('attack Y train shape: ',attack_Y_train.shape)

    attack_X_out_list = []
    for i, (batch_x, batch_y) in enumerate(out_loader):
        feature_vectors = attackdataset_x_1_batch(shadow,batch_x,device=device)
        attack_X_out_list.append(feature_vectors)
    attack_X_out = torch.cat(attack_X_out_list,0)
    print(len(attack_X_out))
    for i in attack_X_out:
      attack_set.append((i.detach(),torch.zeros(1)))
    # out: label = 0
    #attack_Y_out = torch.zeros(attack_X_out.shape[0])
    #print('attack Y out shape: ',attack_Y_out.shape)

    # combine train and out to get the whole dataset's X and Y
    #attack_X = torch.cat([attack_X_train,attack_X_out],0)
    #attack_Y = torch.cat([attack_Y_train,attack_Y_out],0)
    #print('attack X total shape: ',attack_X.shape)
    #return attack_X.detach(),attack_Y
    shuffle(attack_set)
    # check first 10 elements
    print('- - - - - - - - - ')
    print('num of data ')
    print(len(attack_set))
    print('- - - - - - - - - ')
    print('first 10 elements ')
    print(attack_set[0:9])
    return attack_set

### ***Define attack model's dataset class***

In [ ]:
class attack_dataset(Dataset):
    # N data points, features include N top 3 probabilities, labels include N labels(1/0)
    # elements in features and labels are already tensor.
    def __init__(self, attack_set, transform = None, device=torch.device("cpu")):
        self.attack_set = attack_set
        self.device = device
        self.transform = transform

    def __getitem__(self, item):
        # extract x,y , x,y are all tensors
        x = self.attack_set[item][0]
        y = self.attack_set[item][1]
        # send x,y to device
        x = x.to(self.device)
        y = y.to(self.device)
        
        return x, y

    def __len__(self):
        return len(self.attack_set)

## ***Main part***

### ***CIFAR10***

#### *Download, split and load data*

In [ ]:
# prepare original data
transforms = T.Compose([T.ToTensor(),T.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
train_data_CIFAR10 = datasets.CIFAR10(root="./data_1/", train=True,transform=transforms, download=True)
test_data_CIFAR10 = datasets.CIFAR10(root="./data_1/",train=False,transform=transforms, download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data_1/cifar-10-python.tar.gz to ./data_1/
Files already downloaded and verified


In [ ]:
# set the length of each split
length_train_data_CIFAR10 = len(train_data_CIFAR10)
lengthlist = [int(0.25*length_train_data_CIFAR10),int(0.25*length_train_data_CIFAR10),int(0.25*length_train_data_CIFAR10),int(0.25*length_train_data_CIFAR10)]
print(length_train_data_CIFAR10)
print(lengthlist)

50000
[12500, 12500, 12500, 12500]


In [ ]:
# CIFAR10
# split the dataset into 4 part: target train/out, shadow train/out
train_target_dataset_CIFAR10,out_target_dataset_CIFAR10,train_shadow_dataset_CIFAR10,out_shadow_dataset_CIFAR10 = random_split(
    dataset=train_data_CIFAR10,
    lengths=lengthlist,
    generator=torch.Generator().manual_seed(19260817)
)
# This is the main dataloader with the total dataset
target_train_loader = DataLoader(dataset=train_target_dataset_CIFAR10, batch_size=64, shuffle = True)
target_out_loader = DataLoader(dataset=out_target_dataset_CIFAR10, batch_size=64, shuffle = True)

shadow_train_loader = DataLoader(dataset=train_shadow_dataset_CIFAR10, batch_size=64, shuffle = True)
shadow_out_loader = DataLoader(dataset=out_shadow_dataset_CIFAR10, batch_size=64, shuffle = True)

testloader = DataLoader(dataset=test_data_CIFAR10, batch_size=64, shuffle = True)


#### *Pretrain target and shadow models*

In [ ]:
# use train_target to train target model
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train target
# load model
# if use MNIST, inputchannels=1, if use CIFAR10, inputchannels=3, both output classes = 10
# related info could be get by using input_channels=train_data_loader.dataset.channels, output_num=train_data_loader.dataset.class_num
target = basicNet(inputchannels=3, outputclasses=10).to(device)
# settings
criterion = nn.CrossEntropyLoss()
sgd = optim.SGD(target.parameters(), lr=0.01, momentum=0.9)
epoch = 50

# train
print("start training: ")
for i in range(epoch):
    loss_train = train(target, target_train_loader, criterion, sgd)
    # Acc
    acc_train = evaluation(target, target_train_loader, batch_size=64)
    acc_test = evaluation(target, testloader, batch_size=64)
    print("epoch%d   loss: %.5f  training accuracy: %.5f  testing accuracy: %.5f"\
      % (i, loss_train, acc_train, acc_test))
torch.save(target.state_dict(), "./models/target_CIFAR10.pth")

start training: 


196it [00:04, 44.47it/s]


epoch0   loss: 2.30894  training accuracy: 0.11344  testing accuracy: 0.11660


196it [00:04, 44.99it/s]


epoch1   loss: 2.24300  training accuracy: 0.26384  testing accuracy: 0.26410


...

epoch47   loss: 1.69022  training accuracy: 0.79344  testing accuracy: 0.51840


196it [00:04, 44.31it/s]


epoch48   loss: 1.68164  training accuracy: 0.80072  testing accuracy: 0.51970


196it [00:04, 43.76it/s]


epoch49   loss: 1.67669  training accuracy: 0.79688  testing accuracy: 0.52370


In [ ]:
# use train_shadow to train shadow model
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train shadow
# load model
# if use MNIST, inputchannels=1, if use CIFAR10, inputchannels=3, both output classes = 10
# related info could be get by using input_channels=train_data_loader.dataset.channels, output_num=train_data_loader.dataset.class_num
shadow = basicNet(inputchannels=3, outputclasses=10).to(device)
# settings
criterion = nn.CrossEntropyLoss()
sgd = optim.SGD(shadow.parameters(), lr=0.01, momentum=0.9)
epoch = 50

# train
print("start training: ")
for i in range(epoch):
    loss_train = train(shadow, shadow_train_loader, criterion, sgd)
# Acc
acc_train = evaluation(shadow, shadow_train_loader, batch_size=64)
acc_test = evaluation(shadow, testloader, batch_size=64)
print("epoch%d   loss: %.5f  training accuracy: %.5f  testing accuracy: %.5f"\
      % (epoch, loss_train, acc_train, acc_test))
torch.save(shadow.state_dict(), "./models/shadow_CIFAR10.pth")

start training: 


196it [00:04, 44.19it/s]
196it [00:04, 43.58it/s]
196it [00:04, 45.03it/s]
196it [00:04, 44.42it/s]
196it [00:04, 45.00it/s]
196it [00:04, 45.05it/s]
196it [00:04, 45.05it/s]
196it [00:04, 45.52it/s]
196it [00:04, 44.96it/s]
196it [00:04, 44.66it/s]
196it [00:04, 45.12it/s]
196it [00:04, 44.40it/s]
196it [00:04, 45.22it/s]
196it [00:04, 44.90it/s]
196it [00:04, 45.25it/s]
196it [00:04, 43.88it/s]
196it [00:04, 44.68it/s]
196it [00:04, 44.04it/s]
196it [00:04, 44.75it/s]
196it [00:04, 44.66it/s]
196it [00:04, 43.79it/s]
196it [00:04, 44.82it/s]
196it [00:04, 43.51it/s]
196it [00:04, 44.52it/s]
196it [00:04, 43.63it/s]
196it [00:04, 44.51it/s]
196it [00:04, 44.24it/s]
196it [00:04, 44.55it/s]
196it [00:04, 44.76it/s]
196it [00:04, 44.47it/s]
196it [00:04, 43.85it/s]
196it [00:04, 45.20it/s]
196it [00:04, 45.58it/s]
196it [00:04, 45.55it/s]
196it [00:04, 45.46it/s]
196it [00:04, 45.08it/s]
196it [00:04, 45.60it/s]
196it [00:04, 45.32it/s]
196it [00:04, 45.21it/s]
196it [00:04, 44.56it/s]


epoch50   loss: 1.66223  training accuracy: 0.81624  testing accuracy: 0.52080


#### *Load pretrained target and shadow models (to save time)*

In [ ]:
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# load pretrained model
target_pre = basicNet(inputchannels=3, outputclasses=10).to(device)
# if gpu, remove map_location=torch.device('cpu')
target_pre.load_state_dict(torch.load("./models/target_CIFAR10.pth",map_location=torch.device('cpu')))
target_pre.eval()
shadow_pre = basicNet(inputchannels=3, outputclasses=10).to(device)
# if gpu, remove map_location=torch.device('cpu')
shadow_pre.load_state_dict(torch.load("./models/shadow_CIFAR10.pth",map_location=torch.device('cpu')))
shadow_pre.eval()

In [ ]:
# test if these models are loaded
acc_test1 = evaluation(target_pre, testloader, batch_size=64)
print(acc_test1)
acc_test2 = evaluation(shadow_pre, testloader, batch_size=4)
print(acc_test2)

tensor(0.9778)
tensor(0.9750)


In [ ]:
shadow = shadow_pre
target = target_pre

#### *Create dataset for attack model*

In [ ]:
# 1. create train dataset for attack model
# use shadow model and dataset: out_shadow, train_shadow
print('Creating training dataset for attack model:')
train_attack_set = get_attackdataset(shadow,shadow_train_loader,shadow_out_loader,device=device)
train_attack = attack_dataset(attack_set=train_attack_set,device=device)
print('- - - - - - - - - - - - - - - - - - ')
print('Creating test dataset for attack model:')
# 2. create test dataset for attack model
# use target model and dataset: out_target, train_target
test_attack_set = get_attackdataset(target,target_train_loader,target_out_loader,device=device)
test_attack = attack_dataset(attack_set=test_attack_set,device=device)

Creating training dataset for attack model:
12500
12500
- - - - - - - - - 
num of data 
25000
- - - - - - - - - 
first 10 elements 
[(tensor([9.9998e-01, 2.1753e-05, 1.4423e-07], device='cuda:0'), tensor([1.])), (tensor([9.9976e-01, 2.3604e-04, 4.8778e-08], device='cuda:0'), tensor([1.])), (tensor([0.8653, 0.1309, 0.0037], device='cuda:0'), tensor([0.])), (tensor([0.5144, 0.3852, 0.1004], device='cuda:0'), tensor([0.])), (tensor([9.9018e-01, 9.1317e-03, 6.8850e-04], device='cuda:0'), tensor([0.])), (tensor([9.9923e-01, 5.6915e-04, 9.7401e-05], device='cuda:0'), tensor([1.])), (tensor([1.0000e+00, 9.0939e-11, 1.6704e-12], device='cuda:0'), tensor([1.])), (tensor([0.4690, 0.3400, 0.1558], device='cuda:0'), tensor([1.])), (tensor([9.9980e-01, 1.9650e-04, 1.5284e-06], device='cuda:0'), tensor([1.]))]
- - - - - - - - - - - - - - - - - - 
Creating test dataset for attack model:
12500
12500
- - - - - - - - - 
num of data 
25000
- - - - - - - - - 
first 10 elements 
[(tensor([1.0000e+00, 1.967

#### *Attack model 1*

##### *Prepare dataset for attack model 1: LGBMClassifier*

In [ ]:
train_feature = []
train_label = []
for i in train_attack_set:
  train_feature.append(i[0].cpu().numpy())
  train_label.append(int(i[1]))

test_feature = []
test_label = []
for i in test_attack_set:
  test_feature.append(i[0].cpu().numpy())
  test_label.append(int(i[1]))

##### *Attack model 1: LGBMClassifier and output*

In [ ]:
attack_model = lgb.LGBMClassifier(objective='binary', reg_lambda=10, n_estimators=10000)
attack_model.fit(train_feature, train_label)
predict = attack_model.predict(test_feature)
precision_general, recall_general, _, _ = precision_recall_fscore_support(y_pred=predict,y_true=test_label,average="binary")
print('End fitting and predicting')
print('- - - - - - - - - - - - - - - ')

print('result')
print('- - - - - - - - - - - - - - - ')
print('precision: {:.5f}'.format(precision_general))
print('recall: {:.5f}'.format(recall_general))

End fitting and predicting
- - - - - - - - - - - - - - - 
result
- - - - - - - - - - - - - - - 
precision: 0.54766
recall: 0.58232


#### *Attack model 2*

Result is not good. Need more tuning or trick. Guess one possible way is to do some transforms on feature vectors such as log.

##### *Define a MLP class for attack model 2*

In [ ]:
class MLP(nn.Module):
        # 3 features, thus input size=3
        # Yes or no, and use sigmoid thus output size=1
        # If use softmax, output size = 2
        # Loss: cross entropy has contained softmax 
    def __init__(self, num_of_features=3):
        super().__init__()
        self.fc1 = nn.Linear(num_of_features, 64)
        self.fc2 = nn.Linear(64, 2)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Hidden layer
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x,dim=-1)
        return x

##### *Functions for training and evaluating (attack model 2)*

In [ ]:
def attack_train(model, dataloader, criterion, opt):
    running_loss = 0
    # switch to model:train
    # no difference here since no dropout and BN
    model.train()
    count = 0
    for i, data in tqdm(enumerate(dataloader)):
        opt.zero_grad()
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        predict = model(imgs)
        #print(predict.dtype)
        #print(labels.dtype)
        labels = torch.flatten(labels.long())
        #print(predict.shape,labels.shape)
        #print(predict)
        #print(labels)
        loss = criterion(predict, labels)
        loss.backward()
        opt.step()
        count = i
        running_loss += loss
    return running_loss / count

def attack_evaluation(model, dataloader, batch_size=64):
    # switch to model:eval
    # no difference here since no dropout and BN    
    model.eval()
    # y_tensorlist is a list consists of some tensors
    y_true_tensorlist = []
    y_predict_tensorlist = []
    for step, (batch_x, batch_y) in enumerate(dataloader):
        batch_x,batch_y = batch_x.to(device),batch_y.to(device)
        batch_y = torch.flatten(batch_y)
        output = model(batch_x)
        #output = F.softmax(output,dim=-1)
        #print(output)
        # compare cols
        batch_y_predict = torch.argmax(output, dim=1)
        #print(output)        
        #zero = torch.zeros_like(output)
        #one = torch.ones_like(output)
        #batch_y_predict = torch.where(output > 0.5, one, zero)
        #print(batch_y_predict)
        #print(batch_y_predict)
        y_predict_tensorlist.append(batch_y_predict)        
        y_true_tensorlist.append(batch_y)
        
    # combine the tensors in the list into one
    y_true = torch.cat(y_true_tensorlist,0)
    y_predict = torch.cat(y_predict_tensorlist,0)

    # compute accuracy
    length = len(y_true)
    right_length = torch.sum(y_true == y_predict)
    #print(right_length/length)
    
    return right_length/length,y_true,y_predict


##### *Train attack model 2*

In [ ]:
# no gpu, thus device is only cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Train attack
attack = MLP(num_of_features=3).to(device)
# settings
#criterion = nn.BCELoss()
#criterion = nn.NLLLoss()
# crossentropy contains softmax, use other loss
criterion = nn.CrossEntropyLoss()
#opt = optim.SGD(target.parameters(), lr=0.00001, momentum=0.9)
opt = optim.Adam(attack.parameters(), lr=0.001)
epoch = 100
train_attack_dataloader = DataLoader(dataset=train_attack, batch_size=64, shuffle = True)
test_attack_dataloader = DataLoader(dataset=test_attack, batch_size=64, shuffle = True)
# train
print("start training: ")
for i in range(epoch):
    loss_train = attack_train(attack, train_attack_dataloader, criterion, opt)

    print("epoch%d   loss: %.5f "\
      % (i+1, loss_train))
torch.save(target.state_dict(), "./models/attack_MNIST.pth")

start training: 


391it [00:01, 200.94it/s]


epoch1   loss: 0.68153 


391it [00:01, 260.41it/s]


epoch2   loss: 0.66934 


...

epoch98   loss: 0.65786 


391it [00:01, 254.21it/s]


epoch99   loss: 0.65800 


391it [00:01, 255.79it/s]

epoch100   loss: 0.65801 


In [ ]:
_,y_true,y_predict = attack_evaluation(attack, test_attack_dataloader, batch_size=64)

In [ ]:
precision_score(y_true.cpu().numpy(), y_predict.cpu().numpy(), average='binary')

0.5682242990654206

In [ ]:
recall_score(y_true.cpu().numpy(), y_predict.cpu().numpy(), average='binary')

0.80256